In [ ]:
!pip install plotly>=4.7.1
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [ ]:
pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 116 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
#%% Montar librerías base para todo el proceso

import os
import subprocess
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import networkx as nx
from os.path import expanduser, join

import itertools
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import os

import itertools
import plotly.express as px
import pandas as pd
import plotly.io as pio

In [ ]:
execfile('/gdrive/Shareddrives/GENSyR /CNF (Complejidad y NeuroFísica)/Carpeta de trabajo/Tesis Juan y Diego /Resultados FUNCIONES Tesis/Funciones para el analisis/medidas_entropy_energia_grafo.py')

In [ ]:
# Funcion para extraer las medidas globales de un grafo
def medidasglobales(M):

  grafo = nx.convert_matrix.from_numpy_array(M, parallel_edges=False, create_using = nx.Graph())

  Gcc = sorted(nx.connected_components(grafo), key=len, reverse=True)
  grafo = grafo.subgraph(Gcc[0])

  label_medidas = ['Intermediación', 'Energía', 'Cercanía', 'Clusterización', 'Pequeño Mundo']

  entropia = entropia_grafo(grafo)
  energia = energia_grafo(grafo)
  intermediacion = np.mean(np.array(list(nx.betweenness_centrality(grafo, weight = True).values())))
  geodesica = nx.average_shortest_path_length(grafo, weight=True)
  clusterizacion = nx.average_clustering(grafo, weight = True)
  peqmundo = clusterizacion/geodesica
  #eigenvector = nx.eigenvector_centrality(grafo, weight = True, max_iter = 10000000)

  g_distance_dict = {(e1, e2): 1 / weight for e1, e2, weight in grafo.edges(data='weight')}
  nx.set_edge_attributes(grafo, g_distance_dict, 'distance')
  cercania = np.mean(np.array(list(nx.closeness_centrality(grafo, distance = "distance").values())))

  medidas_grupo = [intermediacion, energia, cercania, clusterizacion, peqmundo]

  return medidas_grupo, label_medidas

In [ ]:
tareas = {'WM': 'Memoria Operativa','Emotion': 'Emoción', 'Social': 'Social'}
sexos = {'M': 'Masculino','F': 'Femenino'}
puntuaciones = {'+': 'Puntaje: Alto','-': 'Puntaje: Bajo'}
conectividades = {'EST_sum':'Estructural','SIM_PE':'Simulación', 'REST1_PE':'Reposo', 'SOCIAL_PE':'Social', 'WM_PE':'Memoria Operativa', 'EMOTION_PE':'Emoción'}

In [ ]:
#%% Leer y cargar las rutas de los datos del HCP
home = '/gdrive/Shareddrives/GENSyR /CNF (Complejidad y NeuroFísica)/Carpeta de trabajo/Tesis Juan y Diego /Datos Poblaciones/Sujetos Tesis'

home_figuras = '/gdrive/Shareddrives/GENSyR /CNF (Complejidad y NeuroFísica)/Carpeta de trabajo/Tesis Juan y Diego /Figuras'

In [ ]:
for conectividad in conectividades:

  medidas_grupos = {}

  for tarea in tareas:
    for sexo in sexos:
      for puntuacion in puntuaciones:

        sujeto = tarea + '_' + sexo + puntuacion

        name = conectividad + '_' + sujeto + '.npy'
        fname = join(home, sujeto, name)
        M = np.load(fname)

        if conectividades[conectividad] == 'Estructural':
          M = M/np.max(M)
          M = M*(M > 0.05)
        else:
          M = M/np.max(M)
          M = M*(M > 0.05)

        #print(M.shape)
        medidas_grupo, label_medidas = medidasglobales(M)

        medidas_grupos[tarea + ' ' + sexo + puntuacion] = medidas_grupo

  df = pd.DataFrame(medidas_grupos, index = label_medidas)
  df = df.div(df.max(axis=1), axis=0)
  df = df.append(df.iloc[[0]])

  dfiguras = join(home_figuras, 'analisis global')

  for tarea in tareas:

    # Se verifica si la carpeta de parcelas para cada sujeto existe
    ffigura = join(dfiguras, tarea)
    if not os.path.exists(ffigura):
      os.mkdir(ffigura)

    fig = go.Figure()

    for sexo in sexos:
      for puntuacion in puntuaciones:

        sujeto = tarea + ' ' + sexo + puntuacion

        fig.add_trace(go.Scatterpolar(r = df[sujeto], theta = df.index, name = tareas[tarea] + ' ' + sexo + puntuacion))#, fill='toself'))


    fig.update_layout(title = conectividades[conectividad], showlegend=True, polar = dict(radialaxis = dict(showticklabels=True, tickangle = 90, angle = 25), angularaxis = dict(showticklabels=True)))
    fig.update_layout(font_family="Arial",font_color="black",font_size = 15,title_font_family="Arial",title_font_color="black",legend_title_font_color="red")
    fig.update_xaxes(title_font_family="Arial")
    namefig = 'globales_' + tarea + '_' + conectividad + '.png'
    ffig = join(ffigura, namefig)
    pio.write_image(fig, file = ffig, scale=6, width=600, height=500)

  for sexo in sexos:

    # Se verifica si la carpeta de parcelas para cada sujeto existe
    ffigura = join(dfiguras, sexo)
    if not os.path.exists(ffigura):
      os.mkdir(ffigura)

    fig = go.Figure()

    for tarea in tareas:
      for puntuacion in puntuaciones:

        sujeto = tarea + ' ' + sexo + puntuacion

        fig.add_trace(go.Scatterpolar(r = df[sujeto], theta = df.index, name = tareas[tarea] + ' ' + sexo + puntuacion))#, fill='toself'))


    fig.update_layout(title = conectividades[conectividad], showlegend=True, polar = dict(radialaxis = dict(showticklabels=True, tickangle = 90, angle = 25), angularaxis = dict(showticklabels=True)))
    fig.update_layout(font_family="Arial",font_color="black",font_size = 15,title_font_family="Arial",title_font_color="black",legend_title_font_color="red")
    fig.update_xaxes(title_font_family="Arial")
    namefig = 'globales_' + sexo + '_' + conectividad + '.png'
    ffig = join(ffigura, namefig)
    pio.write_image(fig, file = ffig, scale=6, width=600, height=500)


  for puntuacion in puntuaciones:

    # Se verifica si la carpeta de parcelas para cada sujeto existe
    ffigura = join(dfiguras, puntuacion)
    if not os.path.exists(ffigura):
      os.mkdir(ffigura)

    fig = go.Figure()

    for tarea in tareas:
      for sexo in sexos:

        sujeto = tarea + ' ' + sexo + puntuacion

        fig.add_trace(go.Scatterpolar(r = df[sujeto], theta = df.index, name = tareas[tarea] + ' ' + sexo + puntuacion))#, fill='toself'))


    fig.update_layout(title = conectividades[conectividad], showlegend=True, polar = dict(radialaxis = dict(showticklabels=True, tickangle = 90, angle = 25), angularaxis = dict(showticklabels=True)))
    fig.update_layout(font_family="Arial", font_color="black", font_size = 15, title_font_family="Arial", title_font_color="black", legend_title_font_color="red")
    fig.update_xaxes(title_font_family="Arial")
    namefig = 'globales_' + puntuacion + '_' + conectividad + '.png'
    ffig = join(ffigura, namefig)
    pio.write_image(fig, file = ffig, scale=6, width=600, height=500)
